# im2latex(S): Tokenizer

&copy; Copyright 2017 Sumeet S Singh

    This file is part of im2latex solution by Sumeet S Singh.

    This program is free software: you can redistribute it and/or modify
    it under the terms of the Affero GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    Affero GNU General Public License for more details.

    You should have received a copy of the Affero GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.

In [1]:
import pandas as pd
import os
import re
import codecs
from IPython.display import display
from six.moves import cPickle as pickle
import string

In [2]:
data_dir = '../data/generated2'

#### Characterset Chart for Reference
##### ASCII Control Characters

                        CTRL   (^D means to hold the CTRL key and hit d)
    Oct  Dec Char  Hex  Key     Comments
    \000   0  NUL  \x00  ^@ \0 (Null byte)
    \001   1  SOH  \x01  ^A    (Start of heading)
    \002   2  STX  \x02  ^B    (Start of text)
    \003   3  ETX  \x03  ^C    (End of text) (see: UNIX keyboard CTRL)
    \004   4  EOT  \x04  ^D    (End of transmission) (see: UNIX keyboard CTRL)
    \005   5  ENQ  \x05  ^E    (Enquiry)
    \006   6  ACK  \x06  ^F    (Acknowledge)
    \007   7  BEL  \x07  ^G    (Ring terminal bell)
    \010   8   BS  \x08  ^H \b (Backspace)  (\b matches backspace inside [] only)
                                            (see: UNIX keyboard CTRL)
    \011   9   HT  \x09  ^I \t (Horizontal tab)
    \012  10   LF  \x0A  ^J \n (Line feed)  (Default UNIX NL) (see End of Line below)
    \013  11   VT  \x0B  ^K    (Vertical tab)
    \014  12   FF  \x0C  ^L \f (Form feed)
    \015  13   CR  \x0D  ^M \r (Carriage return)  (see: End of Line below)
    \016  14   SO  \x0E  ^N    (Shift out)
    \017  15   SI  \x0F  ^O    (Shift in)
    \020  16  DLE  \x10  ^P    (Data link escape)
    \021  17  DC1  \x11  ^Q    (Device control 1) (XON) (Default UNIX START char.)
    \022  18  DC2  \x12  ^R    (Device control 2)
    \023  19  DC3  \x13  ^S    (Device control 3) (XOFF)  (Default UNIX STOP char.)
    \024  20  DC4  \x14  ^T    (Device control 4)
    \025  21  NAK  \x15  ^U    (Negative acknowledge)  (see: UNIX keyboard CTRL)
    \026  22  SYN  \x16  ^V    (Synchronous idle)
    \027  23  ETB  \x17  ^W    (End of transmission block)
    \030  24  CAN  \x18  ^X    (Cancel)
    \031  25  EM   \x19  ^Y    (End of medium)
    \032  26  SUB  \x1A  ^Z    (Substitute character)
    \033  27  ESC  \x1B  ^[    (Escape)
    \034  28  FS   \x1C  ^\    (File separator, Information separator four)
    \035  29  GS   \x1D  ^]    (Group separator, Information separator three)
    \036  30  RS   \x1E  ^^    (Record separator, Information separator two)
    \037  31  US   \x1F  ^_    (Unit separator, Information separator one)
    \177 127  DEL  \x7F  ^?    (Delete)  (see: UNIX keyboard CTRL)
    
    string.printable = '0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'
    string.whitespace = '\t\n\x0b\x0c\r '

In [3]:
LATEX_RE = re.compile(r"(?:\\[a-zA-Z]+)|.")


In [4]:
def get_df_clean(data_dir_):
    NONPRINTABLE_CHARS_RE = r'[^\\' + string.printable + r']'
    DELETE_RE = re.compile(r".\x7F")
    PERCENTS_RE = r'%'
    try:
        return pd.read_pickle(os.path.join(data_dir_,'image_details_clean.pkl'))
    except Exception as e:
        print('Exception: ', e)    
        df = pd.read_pickle(os.path.join(data_dir_,'image_details.pkl'))
        # Make sure everything's ascii
        # Coalesce whitespace to a single space
        # Strip whitespace from the sides
        # Strip percent signs from the sides
        # Discard strings with non-printable characters
        # Discard strings with embedded percent signs (because textogif ignores everything after the % sign)
        cleaned = df.latex.str.decode('ascii').str.encode('ascii').str.replace(r"\s+", ' ').str.strip().str.strip('%')
        df = df.assign(latex_ascii=cleaned, latex_ascii_len=cleaned.str.len())
        bad1 = df.latex.str.contains(NONPRINTABLE_CHARS_RE)
        bad2 = df.latex.str.contains(PERCENTS_RE).index
        good = df.index.difference(bad1.union(bad2))
        df = df[good]
        #df.to_pickle((os.path.join(data_dir_,'image_details_clean.pkl')))
        return df

In [5]:
df_clean = get_df_clean(data_dir)
display(df_clean.shape)

('Exception: ', IOError(2, 'No such file or directory'))


(102180, 0)

In [19]:
df_clean.head()

,image,formula_name,latex,formula_len,height,width,latex_ascii,latex_ascii_len
0,450f7c1496143fd_basic.png,450f7c1496143fd_basic,\int_{-\epsilon}^\infty dl\: {\rm e}^{-l\zeta}...,178,78,738,\int_{-\epsilon}^\infty dl\: {\rm e}^{-l\zeta}...,177
1,868d5037af9e4b4_basic.png,868d5037af9e4b4_basic,ds^{2} = (1 - {qcos\theta\over r})^{2\over 1 +...,199,94,962,ds^{2} = (1 - {qcos\theta\over r})^{2\over 1 +...,198
2,af0b6c3ee18804a_basic.png,af0b6c3ee18804a_basic,\widetilde\gamma_{\rm hopf}\simeq\sum_{n>0}\wi...,89,87,291,\widetilde\gamma_{\rm hopf}\simeq\sum_{n>0}\wi...,89
3,dda45eca6d32fa3_basic.png,dda45eca6d32fa3_basic,"({\cal L}_a g)_{ij} = 0, \ \ \ \ ({\cal L}_a H...",59,35,405,"({\cal L}_a g)_{ij} = 0, \ \ \ \ ({\cal L}_a H...",59
4,67eb249ed1c20d2_basic.png,67eb249ed1c20d2_basic,S_{stat} = 2\pi \sqrt{N_5^{(1)} N_5^{(2)} N_5^...,100,60,521,S_{stat} = 2\pi \sqrt{N_5^{(1)} N_5^{(2)} N_5^...,100


In [20]:
def get_vocabulary():
    try:
        return pd.read_pickle(os.path.join(data_dir_,'vocabulary.pkl'))
    except Exception as e:
        

IndentationError: expected an indented block (<ipython-input-20-4a47b288ff3b>, line 5)

In [21]:
s_token = df_clean.latex_ascii.str.findall(LATEX_RE)
s_token.head()

0    [\int, _, {, -, \epsilon, }, ^, \infty,  , d, ...
1    [d, s, ^, {, 2, },  , =,  , (, 1,  , -,  , {, ...
2    [\widetilde, \gamma, _, {, \rm,  , h, o, p, f,...
3    [(, {, \cal,  , L, }, _, a,  , g, ), _, {, i, ...
4    [S, _, {, s, t, a, t, },  , =,  , 2, \pi,  , \...
Name: latex_ascii, dtype: object

In [22]:
vocabulary = set([])
s_token.agg(lambda l: vocabulary.update(l))
len(vocabulary)

781

In [23]:
vocabulary

{' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 '\\',
 '\\A',
 '\\AA',
 '\\AC',
 '\\B',
 '\\BF',
 '\\BG',
 '\\BJ',
 '\\BP',
 '\\Bbb',
 '\\Big',
 '\\Bigg',
 '\\Biggl',
 '\\Biggm',
 '\\Biggr',
 '\\Bigl',
 '\\Bigm',
 '\\Bigr',
 '\\Box',
 '\\C',
 '\\CF',
 '\\CS',
 '\\D',
 '\\DE',
 '\\DF',
 '\\DV',
 '\\Delta',
 '\\Department',
 '\\Downarrow',
 '\\Dslash',
 '\\E',
 '\\F',
 '\\Flows',
 '\\G',
 '\\Ga',
 '\\Gamma',
 '\\H',
 '\\Huge',
 '\\I',
 '\\ISO',
 '\\Im',
 '\\J',
 '\\K',
 '\\L',
 '\\LARGE',
 '\\Lambda',
 '\\Large',
 '\\Leftrightarrow',
 '\\Li',
 '\\Longleftarrow',
 '\\Longleftrightarrow',
 '\\Longrightarrow',
 '\\M',
 '\\N',
 '\\O',
 '\\Od',
 '\\Omega',
 '\\P',
 '\\Pcm',
 '\\Phi',
 

In [24]:
s_token.shape

(102180,)

In [32]:
string.printable

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [33]:
string.whitespace

'\t\n\x0b\x0c\r '

In [44]:
string.whitespace

'\t\n\x0b\x0c\r '

In [75]:
PRINTABLE_MATCH_RE = r"^[\\" + string.printable + r']+$'
print PRINTABLE_MATCH_RE
NON_PRINTABLE_RE = r'[^\\' + string.printable + r']'
print NON_PRINTABLE_RE

^[\\0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	
]+$
[^\\0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	
]


In [76]:
sum(df_clean.latex.str.match(PRINTABLE_RE))

102174

In [153]:
for i in df_clean[df_clean.latex.str.contains(NON_PRINTABLE_RE)].latex.index:
    display(df_clean.latex[i])

'\\label{vvvfq}\\xi_q:=\\delta_v q^i(t) \\frac{\\partial}{\\partial q^i},% \x7f'

'\\label{lied1}\\delta_h F(q^i(t), \\dot q^i(t), t)={L}_\\xi F(q^i(t), \\dot q^i(t), t).%  \x7f'

'\\label{dSD}d_v S_D=\\sum_k (t_{k+1}-t_k)d_v {L_D}^{(k)}, \\qquad d_v{L_D}^{(k)}={{\\cal E}_D}^{(k)}+\\Delta_k{\\theta_D}^{(k)},%\x7f'

'\\label{ddsDf}d_v{{\\cal E}_D}^{(i,j)}+\\Delta_\\mu {\\omega_D}^{\\mu (i,j)}=0,\\qquad{\\omega_D}^{\\mu(i,j)}:=d_v{\\theta_D}^{\\mu(i,j)}.%=d_v{p_{i}}^{(k+1)}\\wedge d_vq^{i(k)}.% .  \x7f'

'\\label{eleqm}\\frac{\\partial {L}}{\\partial q^i}-\\frac{d}{dt}(\\frac{\\partial{L}}{\\partial \\dot q^i})=0.%  \x7f'

'\\label{dDt}\\delta_t(t_{k+1}-t_k)=\\Delta_k(\\delta_tt_k)(t_{k+1}-t_k),%  \x7f'

In [83]:
sum(df_clean.latex.str.contains('\x7f'))

6

In [139]:
BAD_SYNTAX_MATCH_RE = r'^[^\s]+.*%.*[^\s]+$'
df_percent = df_clean[df_clean.latex_ascii.str.contains(BAD_SYNTAX_MATCH_RE)]
print df_percent.shape
df_percent.latex.head()

(2575, 8)


71     \label{gg3}Q^M=\dot g_0\int_{\phi (\Sigma )}\s...
81     {\cal{Z}}(\tau{})=\sum_{m}\int{\cal{D}}\Omega{...
147    \exp \left( ip_{\mu }X^{\mu }\right) \rightarr...
148    (\partial_{\gamma},\partial_{\gamma})_{(10)}=\...
159    \delta _{A}\widehat{\phi }\smallskip (\widehat...
Name: latex, dtype: object

In [140]:
BAD_SYNTAX_MATCH_RE = r'%'
df_percent2 = df_clean[df_clean.latex_ascii.str.contains(BAD_SYNTAX_MATCH_RE)]
print df_percent2.shape

(2579, 8)


In [142]:
df_percent2.index.difference(df_percent.index)

Int64Index([49012, 52170, 97018, 99894], dtype='int64')

In [144]:
for i in df_percent2.index.difference(df_percent.index):
    print i, ') ', df_clean.loc[i].latex

 49012 )  \label{vvvfq}\xi_q:=\delta_v q^i(t) \frac{\partial}{\partial q^i},% 
52170 )  \label{lied1}\delta_h F(q^i(t), \dot q^i(t), t)={L}_\xi F(q^i(t), \dot q^i(t), t).%  
97018 )  \label{eleqm}\frac{\partial {L}}{\partial q^i}-\frac{d}{dt}(\frac{\partial{L}}{\partial \dot q^i})=0.%  
99894 )  \label{dDt}\delta_t(t_{k+1}-t_k)=\Delta_k(\delta_tt_k)(t_{k+1}-t_k),%  
